# **ASIRRA base notebook for AI for Astrophysics AAIF Doctoral Course**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/AI_astro_ED_AAIF/blob/main/codes/CNN/classification/ASIRRA/ASIRRA_CIANNA.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

### **CIANNA installation**

#### Query GPU allocation and properties

If nvidia-smi fail, it might indicate that you launched the colab session whithout GPU reservation.  
To change the type of reservation go to "Runtime"->"Change runtime type" and select "GPU" as your hardware accelerator.

In [ ]:
%%shell

nvidia-smi

cd /content/

git clone https://github.com/NVIDIA/cuda-samples/

cd /content/cuda-samples/Samples/1_Utilities/deviceQuery/

make SMS="50 60 70 80"

./deviceQuery | grep Capability | cut -c50- > ~/cuda_infos.txt
./deviceQuery | grep "CUDA Driver Version / Runtime Version" | cut -c57- >> ~/cuda_infos.txt

cd ~/

If you are granted a GPU that supports high FP16 compute scaling (e.g the Tesla T4), it is advised to change the mixed_precision parameter in the prediction to "FP16C_FP32A".  
See the detail description on mixed precision support with CIANNA on the [Systeme Requirements](https://github.com/Deyht/CIANNA/wiki/1\)-System-Requirements) wiki page.

#### Clone CIANNA git repository

In [ ]:
%%shell

cd /content/

git clone https://github.com/Deyht/CIANNA

cd CIANNA

#### Compiling CIANNA for the allocated GPU generation

There is no guaranteed forward or backward compatibility between Nvidia GPU generation, and some capabilities are generation specific. For these reasons, CIANNA must be provided the platform GPU generation at compile time.
The following cell will automatically update all the necessary files based on the detected GPU, and compile CIANNA.

In [ ]:
%%shell

cd /content/CIANNA

mult="10"
cat ~/cuda_infos.txt
comp_cap="$(sed '1!d' ~/cuda_infos.txt)"
cuda_vers="$(sed '2!d' ~/cuda_infos.txt)"

lim="11.1"
old_arg=$(awk '{if ($1 < $2) print "-D CUDA_OLD";}' <<<"${cuda_vers} ${lim}")

sm_val=$(awk '{print $1*$2}' <<<"${mult} ${comp_cap}")

gen_val=$(awk '{if ($1 >= 80) print "-D GEN_AMPERE"; else if($1 >= 70) print "-D GEN_VOLTA";}' <<<"${sm_val}")

sed -i "s/.*arch=sm.*/\\t\tcuda_arg=\"\$cuda_arg -D CUDA -D comp_CUDA -lcublas -lcudart -arch=sm_$sm_val $old_arg $gen_val\"/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" src/python_module_setup.py

./compile.cp CUDA PY_INTERF

mv src/build/lib.linux-x86_64-* src/build/lib.linux-x86_64


**IMPORTANT NOTE**   
CIANNA is mainly used in a script fashion and was not designed to run in notebooks. Every cell code that directly invokes CIANNA functions must be run as a script to avoid possible errors.  
To do so, the cell must have the following structure.

```
%%shell

cd /content/CIANNA

python3 - <<EOF

[... your python code ...]

EOF
```

This syntax allows one to easily edit python code in the notebook while running the cell as a script. Note that all the notebook variables can not be accessed by the cell in this context.


### **ASIRRA**

The ASIRRA (Animal Species Image Recognition for Restricting Access) is a dataset that was originaly used for CAPTCHA and HIP (Human Interactive Proofs).

The dataset comprises 25000 images of variable resolution (averaging around 350x500) and perfectly distributed over the two classes "Cat" and "Dog". For this course, we provide a resized to 128x128 and squared version of the dataset so it can fit into the limited amount of Colab RAM more easily.

#### Downloading and visualizing the data


In [ ]:
%%shell

cd /content/

#Manually upload the directory to github if not yet opened
git clone https://github.com/Deyht/AI_astro_ED_AAIF

In [ ]:
%%shell

cd /content/AI_astro_ED_AAIF/codes/CNN/ASIRRA/

python3 - <<EOF

#Will download the dataset at the fist call
from aux_fct import *

init_data_gen(0)

print("\nOrdered validation examples")
create_val_batch()

print("Create visualization of the validation dataset")
visual_val(8,4)

EOF

In [ ]:
%cd /content/AI_astro_ED_AAIF/codes/CNN/ASIRRA/
from PIL import Image
import matplotlib.pyplot as plt

im = Image.open("val_mosaic.jpg")
plt.figure(figsize=(8,4), dpi=200)
plt.imshow(im)
plt.gca().axis('off')
plt.show()

#### Training a network



In [ ]:

%%shell

cd /content/AI_astro_ED_AAIF/codes/CNN/ASIRRA/

python3 - <<EOF


import numpy as np
from threading import Thread
from aux_fct import *
import gc, sys, glob


#Comment to access system wide install
sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn


def data_augm():
	input_data, targets = create_train_batch()
	cnn.delete_dataset("TRAIN_buf", silent=1)
	cnn.create_dataset("TRAIN_buf", nb_images_per_iter, input_data[:,:], targets[:,:], silent=1)
	return

#Might need more iter to converge depending on other hyperparameter
total_iter = 1000
nb_iter_per_augm = 1
if(nb_iter_per_augm > 1):
	shuffle_frequency = 1
else:
	shuffle_frequency = 0

load_iter = 0
if (len(sys.argv) > 1):
	load_iter = int(sys.argv[1])

start_iter = int(load_iter / nb_iter_per_augm)

cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=3, out_dim=nb_class,
		bias=0.1, b_size=VAL, comp_meth='C_CUDA', dynamic_load=1,
		mixed_precision="FP32C_FP32A", adv_size=30)

init_data_gen()

input_val, targets_val = create_val_batch()
cnn.create_dataset("VALID", nb_keep_val, input_val[:,:], targets_val[:,:])
cnn.create_dataset("TEST", nb_keep_val, input_val[:,:], targets_val[:,:])
del (input_val, targets_val)
gc.collect()

input_data, targets = create_train_batch()
cnn.create_dataset("TRAIN", nb_images_per_iter, input_data[:,:], targets[:,:])

if(load_iter > 0):
	cnn.load("net_save/net0_s%04d.dat"%load_iter, load_iter, bin=1)
else:

	#Build your network architecture here


for run_iter in range(start_iter,int(total_iter/nb_iter_per_augm)):

	t = Thread(target=data_augm)
	t.start()

	#Edit optimization parameters
	cnn.train(nb_iter=nb_iter_per_augm, learning_rate=VAL, end_learning_rate=VAL, shuffle_every=shuffle_frequency ,\
			 control_interv=20, confmat=1, momentum=VAL, lr_decay=VAL, save_every=50,\
			 silent=0, save_bin=1, TC_scale_factor=8.0)

	if(run_iter == start_iter):
		cnn.perf_eval()

	t.join()
	cnn.swap_data_buffers("TRAIN")


EOF


In [ ]:

%%shell

cd /content/AI_astro_ED_AAIF/codes/CNN/ASIRRA/

python3 - <<EOF

import numpy as np
from threading import Thread
from aux_fct import *
import gc, time, sys, glob

#Comment to access system wide install
sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn

load_epoch = 0 #Specify the epoch to load here (base on save_every parameter in cnn.train)

#Change image test mode in aux_fct to change network resolution in all functions
init_data_gen(test_mode=1)

cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=3, out_dim=nb_class,
	bias=0.1, b_size=16, comp_meth='C_CUDA', dynamic_load=1,
	mixed_precision="FP32C_FP32A", adv_size=30, inference_only=1)

#Compute on only half the validation set to reduce memory footprint
input_test, targets_test = create_val_batch()
cnn.create_dataset("TEST", nb_keep_val, input_test[:,:], targets_test[:,:])

cnn.load("net_save/net0_s%04d.dat"%load_epoch, load_epoch, bin=1)

cnn.forward(repeat=1, no_error=1, saving=2, drop_mode="AVG_MODEL")

start = time.perf_counter()
cnn.forward(no_error=1, saving=2, drop_mode="AVG_MODEL")
end = time.perf_counter()

cnn.perf_eval()

raw_pred = np.fromfile("fwd_res/net0_%04d.dat"%(load_epoch), dtype="float32")
pred = np.reshape(raw_pred, (nb_keep_val,-1))

correct = np.shape(np.where(np.argmax(pred[:,:nb_class], axis=1) == np.argmax(targets_test[:,:], axis=1)))[1]
error_rate = 100.0-(correct/nb_keep_val)*100.0

print("Error rate: ", error_rate, "%")
print("Inference time: ", end-start, " s")

EOF